# Data Processing

behaviors.tsv 안의 단일 impression 데이터가 갖는 구조를 잠시 살펴보겠습니다. <br/>
(https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md) <br/>
 <br/>
 Impression ID: `int` <br/>
 User ID: `str` <br/>
 Time: `str` <br/>
 History: `[News_ID, News_ID, News_ID ...]` <br/>
 Impressions: `['News_ID-0', 'News_ID-1', ...]` <br/>
 <br/>
그리고 하나의 뉴스가 갖는 데이터 구조는 다음과 같습니다. <br/>
 <br/>
News ID: `str` <br/>
Category: `str` <br/>
SubCategory: `str` <br/>
Title: `str` <br/>
Abstract: `str` <br/>
(그 외 URL, Title Entities, Abstract Entities - 사용하지 않음) <br/>
 <br/>
이 프로젝트에서는 Title, Abstract의 모든 단어(token)를 인덱스로 치환합니다. <br/>
(어떤 식으로 치환하는지는 word2int를 참고해주세요.) <br/>
그리고 모든 단어는 사전 학습된 GloVe의 embedding lookup table에서 단어에 해당하는 임베딩 벡터를 가져와서 embedding_weights에 저장합니다. <br/>
(GloVe word embedding 파일(약 8GB)에는 약 800만개 가량의 단어와, 해당 단어를 300차원 임베딩 벡터로 변환한 데이터가 저장되어 있습니다.) <br/>
그리고 word2int로 저장되는 인덱스는 embedding_weights의 맨 첫번째 줄의 padding을 제외하고 모두 1대1로 대응됩니다. <br/>
즉 embedding_weights를 불러왔을 때 5번 인덱스의 벡터는 word2int의 5번 인덱스에 해당하는 단어의 임베딩 벡터가 됩니다. <br/>
 <br/>
word2int에는 0번 인덱스가 없습니다. 왜냐하면 0번 인덱스는 모든 문장의 token 길이를 동일하게 맞추는 과정에서 생기는 공백을 매꾸는 데 쓰이기 때문입니다. (padding) <br/>
 <br/>
(이로 인해 생기는 문제가 하나 있는데, embedding_weights 맨 첫번째 줄에 padding용 300차원 영벡터를 추가적으로 생성하지 않은 초기 버전 프로젝트는 <br/>
word2int 맨 마지막 인덱스에 해당하는 단어가 포함된 뉴스의 데이터를 불러올 때 out of bounds 에러가 떴습니다.) <br/>
 <br/>
(`자세한 설명`: word2int에 저장된 모든 단어의 수를 max_word_num이라고 할 때, <br/>
word2int에 매핑된 단어들의 index 범위: 1 ~ max_word_num <br/>
embedding_weights에 저장된 임베딩 벡터의 수: max_word_num <br/>
embedding_weights를 불러왔을 때 생성되는 데이터의 index 범위: 0 ~ (max_word_num-1) <br/>
따라서 word2int의 index를 그대로 사용할 경우, <br/>
데이터에 마지막 인덱스에 해당하는 단어의 임베딩 벡터를 불러오려 하면 오류가 발생합니다. <br/>
그런데 테스트에 주로 사용하는 SENTIREC, NRMS 모델은 Abstract 데이터를 사용하지 않기 때문에, 마지막 인덱스 단어가 Abstract에만 있을 경우 해당 오류가 뜨지 않습니다. <br/>
즉 어떤 데이터셋을 사용하였느냐, 전처리가 어떤 순서로 진행되었느냐에 따라 발생 유무, 타이밍이 오락가락 한다는 것입니다. <br/>
또한 test의 embedding_weights가 모든 데이터의 단어를 포함하며, test 데이터셋을 마지막으로 처리하는 특성상 train 과정에서는 오류가 발생하지 않습니다. <br/>
그리고 가장 큰 문제는 임베딩 벡터가 한 줄씩 밀려서, 불러온 값이 아예 다른 단어의 임베딩 벡터가 된다는 점입니다. <br/>
물론 지금은 전처리 과정에서 embedding_weights 파일 첫 줄에 padding 벡터를 추가해주기 때문에 이러한 문제가 발생하지 않습니다.) <br/>
 <br/>
이러한 인덱스 치환의 결과는 parsed_news.tsv 파일에서 확인할 수 있습니다. <br/>
해당 파일에 저장하는 데이터는 다음과 같이 구성됩니다. <br/>
 <br/>
`News ID, Category, SubCategory, Title, Abstract, VADER Sentiment Score, BERT Sentiment Score`  <br/>
 <br/>
여기서 Category, SubCategory는 category2int.tsv에 매핑된 인덱스에 맞게 치환되고 <br/>
Title, Abstract는 전처리 과정에서 설정한 max_title, max_abstract 값에 맞게 token 길이가 고정됩니다. <br/>
즉 너무 길면 잘리고, 너무 짧으면 padding(index = 0)으로 채워집니다. <br/>
 <br/>
behaviors.tsv의 경우, train 데이터셋은 train_behavior.tsv와 val_behavior.tsv, test 데이터셋은 test_behavior.tsv로 변환됩니다. <br/>
구성은 모두 동일하나, 모델의 학습/테스트 과정에서 사용처가 다릅니다. <br/>
_behavior.tsv 파일의 데이터 구성은 다음과 같습니다. <br/>
 <br/>
`User ID, History, Impressions, Labels` <br/>
 <br/>
공통: User ID는 user2int.tsv에 매핑된 인덱스로 변환되며, Impressions의 클릭 유무를 나타내는 모든 Label이 따로 떼어집니다. <br/>
train_behavior.tsv와 val_behavior.tsv의 경우: <br/>
 negative sampling이 적용되어 positive sample(label = 1) 하나당 n_negative로 설정한 개수의 negative sample(label = 0)만 사용합니다. <br/>
 또한 positive sample이 여러개일 경우, 한 Impression 데이터를 여러 줄로 쪼개어 사용합니다. <br/>
반면 test_behavior.tsv는 negative sampling을 하지도 않고, 한 Impression 데이터를 여러 줄로 쪼개지도 않습니다. <br/>

## Global Settings and imports

In [1]:
# jupyter notebook에서 import 해서 쓰는 모듈의 코드가 변경될 시, 변동 사항을 자동으로 반영해주는 기능 켜기
%load_ext autoreload
%autoreload 2

In [2]:
"""
module import를 진행할 시, sys.path에 등록된 경로에서 해당 모듈 파일을 찾습니다.
그런데 프로젝트 폴더가 기본적으로 등록되어있지 않아서
project/data 같은 경로의 모듈을 사용하기 위해 둘 중 한가지 방법을 써야 합니다.
직접 경로를 추가 => sys.path.append(...)
import할 때 상대 경로 사용 => from ...data.preprocess import
그런데 상대 경로가 이래저래 요상한 점이 많아서
절대 경로를 등록해서 사용하기로 했습니다.
"""
import os
import sys
from os import path

PROJECT_DIR = path.abspath(path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_DIR)
print(PROJECT_DIR)

from data.preprocess.prep_dataset import PrepDataset, PrepDatasetArgs

c:\Users\USER\Desktop\projects\python\newsrecommend\SentiRecTest\project


c:\Users\USER\anaconda3\envs\newsrec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
args = PrepDatasetArgs(
    size = "demo",
    split_test_size = 0.1,
    n_negative = 4,
    max_title = 20,
    max_abstract = 50
)

prep_dataset = PrepDataset(args)

## 1. Train/Test 데이터셋 behaviors.tsv 전처리

### 생성되는 파일
##### Train
1. train_behavior.tsv
2. user2int.tsv
3. val_behavior.tsv
##### Test
1. test_behavior.tsv

## 2. Train/Test 데이터셋 news.tsv 전처리 (통합됨)

### 생성되는 파일
##### Train
1. parsed_news.tsv
2. category2int.tsv
3. embedding_weights.csv
4. word2int.tsv
##### Test
1. parsed_news.tsv
2. embedding_weights.csv
3. word2int.tsv

In [4]:
prep_dataset.pre_processing_dataset()

preparing training data


100%|██████████| 19830/19830 [00:00<00:00, 42158.04it/s]


preparing eval data


100%|██████████| 4880/4880 [00:00<00:00, 2436981.01it/s]


preparing eval data


...

100%|██████████| 2196017/2196017 [00:01<00:00, 1263508.77it/s]
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


preparing/processing train news content


Device set to use cuda:0
100%|██████████| 26740/26740 [02:13<00:00, 200.29it/s]
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


preparing/processing test news content


Device set to use cuda:0
100%|██████████| 18723/18723 [01:30<00:00, 207.46it/s]
